In [ ]:
import requests
import pandas as pd
import urllib
import sys
import time

In [ ]:
country_code = {"ar": "Argentina", "at": "Austria", "au":"Australia", "br":"Brazil", "cl": "Chile", "de":"Germany", "es":"Spain", "fr":"France", "gb":"United Kingdom", "ge":"Georgia", "gr":"Greece", "it":"Italy", "pt":"Portugal", "us": "USA" }
type_code = {"1":"Red", "2":"White", "3":"Sparkling", "4": "Rose", "7": "Dessert", "24":"Fortified"}

In [ ]:
def get_wine_data(wine_id, year, page):
    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0",
    }

    api_url = "https://www.vivino.com/api/wines/{id}/reviews?per_page=50&year={year}&page={page}"

    data = requests.get(
        api_url.format(id=wine_id, year=year, page=page), headers=headers
    ).json()


    return data

In [ ]:
old_links = pd.read_csv("/kaggle/input/wine-link-old/new_wines_links_1.csv")
old_links = old_links["0"].to_list()

In [ ]:
list_wines = []
price_max = "1500"
price_min = "500"
for i in range(1, 90):
    try:
        r = requests.get(
            "https://www.vivino.com/api/explore/explore",
            params={
                "country_code": "IT",
                "country_codes[]": "it",
                "currency_code": "EUR",
                "grape_filter": "varietal",
                "min_rating": "1",
                #"order_by": "ratings_average",
                "order_by": "price",
                "order": "asc",
                #"order": "asc",
                "page": i,
                "price_range_max": price_max,
                "price_range_min": price_min,
                "region_ids[]": 394,
            },
            headers={
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0"
            },
        )

        if not r.json()["explore_vintage"]:
            break

        results = [
            (
                t["vintage"]["wine"]["winery"]["name"],
                t["vintage"]["year"],
                t["vintage"]["wine"]["id"],
                f'{t["vintage"]["wine"]["name"]} {t["vintage"]["year"]}',
                t["vintage"]["statistics"]["ratings_average"],
                t["vintage"]["statistics"]["ratings_count"],
                type_code[str(t["vintage"]["wine"]["type_id"])],
                f'{t["vintage"]["wine"]["region"]["country"]["name"]} {t["vintage"]["wine"]["region"]["name"]}',
                t["vintage"]["wine"]["region"]["country"]["code"],
                f'{t["vintage"]["wine"]["region"]["country"]["most_used_grapes"][0]["name"]} {t["vintage"]["wine"]["region"]["country"]["most_used_grapes"][1]["name"]}',
                round(t["price"]["amount"], 2) if t["price"] is not None else "-",
                f'{"https://www.vivino.com/IT/en/"}{t["vintage"]["seo_name"]}{"/w/"}{t["vintage"]["wine"]["id"]}'
            )
            for t in r.json()["explore_vintage"]["matches"]
        ]
        dataframe = pd.DataFrame(
            results,
            columns=["Winery", "Year", "Wine ID", "Wine", "Rating", "num_review", "Wine type", "Wine region", "Country", "Grape", "price", "link"],
        )
    except Exception:
        print(f"Global exception occured on {i}")
        original_stdout = sys.stdout

        with open('error_page.txt', 'a') as f:
            sys.stdout = f 
            print(f"Global exception occured on {i}")
            sys.stdout = original_stdout
        continue
    ratings = []
    for _, row in dataframe.iterrows():
        page = 1
        while True:
            if row["link"] in old_links: break
            if page % 50 == 0 or page == 1: print(f'Getting info about wine {row["Wine"]}-{row["Year"]} Page {page} Price {row["price"]}')
            iters = 0  
            flag = 0
            while iters < 3:
                try:
                    d = get_wine_data(row["Wine ID"], row["Year"], page)
                    if not d["reviews"]:
                        flag = 1
                        break
                    for r in d["reviews"]:
                        #print(r["vintage"]["seo_name"])
                        if ("statistics" not in r["user"]) and ("vintage" not in r):
                            ratings.append(
                            [
                                row["Year"],
                                row["Wine ID"],
                                r["rating"],
                                r["note"],
                                r["created_at"],
                                r["user"]["id"],
                                "-",
                                "-",
                                "-",
                                r["language"],
                                "-"
                            ]
                        )
                            continue
                        if ("statistics" not in r["user"]):
                            ratings.append(
                            [
                                row["Year"],
                                row["Wine ID"],
                                r["rating"],
                                r["note"],
                                r["created_at"],
                                r["user"]["id"],
                                "-",
                                "-",
                                "-",
                                r["language"],
                                r["vintage"]["seo_name"]
                            ]
                        )
                            continue
                        if ("vintage" not in r):
                            ratings.append(
                            [
                                row["Year"],
                                row["Wine ID"],
                                r["rating"],
                                r["note"],
                                r["created_at"],
                                r["user"]["id"],
                                r["user"]["statistics"]["followers_count"],
                                r["user"]["statistics"]["followings_count"],
                                r["user"]["statistics"]["ratings_count"],
                                r["language"],
                                "-"
                            ]
                        )
                            continue
                            
                            
                        ratings.append(
                            [
                                row["Year"],
                                row["Wine ID"],
                                r["rating"],
                                r["note"],
                                r["created_at"],
                                r["user"]["id"],
                                r["user"]["statistics"]["followers_count"],
                                r["user"]["statistics"]["followings_count"],
                                r["user"]["statistics"]["ratings_count"],
                                r["language"],
                                r["vintage"]["seo_name"]
                            ]
                        )
                    break
                except Exception:
                    iters += 1
                    link = row["link"]
                    print(f"Exception occured on {page}, {link}, num of global page is {i}, num of iters {iters}")
                    original_stdout = sys.stdout  # Save a reference to the original standard output

                    with open('error_page.txt', 'a') as f:
                        sys.stdout = f  # Change the standard output to the file we created.
                        #print(dataframe["links"].drop_duplicates())
                        print(f"Exception occured on {page}, {link}, num of global page is {i}, num of iters {iters}")
                        sys.stdout = original_stdout
                        time.sleep(1)

            page += 1
            if flag == 1: break
            


    ratings = pd.DataFrame(
        ratings, columns=["Year", "Wine ID", "User's Grade", "Note", "CreatedAt", "user id", "followers", "following", "User Ratings", "language", "label"]
    )

    df_out = dataframe.merge(ratings)
    string = "wines_" + str(i) + ".csv"
    list_wines.append(df_out)
    df_out.to_csv(string, index=False, sep = ';')
    if i % 5 == 0 or i == 1: print("Completed ",i)
string = "wines" + price_min +  "-" + price_max + ".csv"
result = pd.concat(list_wines)
result.to_csv(string, index=False, sep = ';')
        
        

   